In [1]:
print("Hello world1")

Hello world1


In [2]:
import os

In [3]:
%pwd

'/workspaces/Text-Summarizer-using-Hugging-face-and-Transformers/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/workspaces/Text-Summarizer-using-Hugging-face-and-Transformers'

In [6]:
from src.textSummarizer.logging import logger
from src.textSummarizer.constants import *
from dataclasses import dataclass
from pathlib import Path

# created entity with dataclass
@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [7]:
# build configuration manager
from src.textSummarizer.utils.common import read_yaml,create_directories
from src.textSummarizer.constants import *

class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)->DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config
     

In [8]:
# build Data transformation component

## external packages
import os 
from transformers import AutoTokenizer
from datasets import load_from_disk


## from src
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import *


class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
                }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched= True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

/workspaces/Text-Summarizer-using-Hugging-face-and-Transformers/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation_comp = DataTransformation(data_transformation_config)
data_transformation_comp.convert()
# data_transformation_comp.convert_examples_to_features()


[2026-01-21 09:36:31 | INFO | summarizerLogger | common.py:60 | Created directory at: artifacts]
[2026-01-21 09:36:31 | INFO | summarizerLogger | common.py:60 | Created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/workspaces/Text-Summarizer-using-Hugging-face-and-Transformers/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 113292.19 examples/s]


TypeError: DataTransformation.convert_examples_to_features() missing 1 required positional argument: 'example_batch'